## Loading Reflectance

In [3]:
import array
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from termcolor import colored
import time
import xarray as xr

**Functions**

In [4]:
def fromstring(x):
    
    '''Converts string to datetime object'''
    
    try:
        d = datetime.datetime.strptime(x, '%d/%m/%Y').date()
    except:
        d = datetime.datetime.strptime(x, '%Y-%m-%d').date()
        
    return d

def fromjulian(x):
    
    '''Converts julian to datetime object'''

    return datetime.datetime.strptime(x, '%Y%j').date()

def compute_ndvi_float(b1: float, b2:float):
    
    if ((b2 + b1) == 0):
        ndvi = 0
    else:
        ndvi = (b2 - b1)/(b2 + b1)
        
    return ndvi

**Loading**

In [5]:
# loading ndvi_MOD data from csv
ref_MOD = pd.read_csv(
    'data/MOD09A1-MOD09A1-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MOD09A1_006_sur_refl_b01', 
               'MOD09A1_006_sur_refl_b02',
               'MOD09A1_006_sur_refl_day_of_year'],
     dtype = {'MOD09A1_006_sur_refl_day_of_year': int})


#renaming the columns
ref_MOD = ref_MOD.rename(columns={"MOD09A1_006_sur_refl_b01": "B1", 
                        "MOD09A1_006_sur_refl_b02": "B2",
                        'MOD09A1_006_sur_refl_day_of_year': 'Composite_date'})


# Convert string Date to datetime.date
ref_MOD['Date'] = ref_MOD['Date'].apply(fromstring)

#Cropping to right dates
a = (ref_MOD['Date']>datetime.date(2002,7,4)).values
ref_MOD = ref_MOD[a]

# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ref_MOD)):
    
    d = ref_MOD['Date'][i]
    c = ref_MOD['Composite_date'][i]
    
    if c != 65535:    #because nd is 65535
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(True)
    elif (compo[i]==compo[i-1]):
        same_compo.append(False)
    else:
        same_compo.append(True)
            
ref_MOD['Composite_date'] = compo   
ref_MOD = ref_MOD[same_compo]

In [6]:
# loading ndvi_MOD data from csv
ref_MYD = pd.read_csv(
    'data/MYD09A1-MYD09A1-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MYD09A1_006_sur_refl_b01', 
               'MYD09A1_006_sur_refl_b02',
               'MYD09A1_006_sur_refl_day_of_year'],
     dtype = {'MYD09A1_006_sur_refl_day_of_year': int})


#renaming the columns
ref_MYD = ref_MYD.rename(columns={"MYD09A1_006_sur_refl_b01": "B1", 
                        "MYD09A1_006_sur_refl_b02": "B2",
                        'MYD09A1_006_sur_refl_day_of_year': 'Composite_date'})


# Convert string Date to datetime.date
ref_MYD['Date'] = ref_MYD['Date'].apply(fromstring)

# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ref_MYD)):
    
    d = ref_MYD['Date'][i]
    c = ref_MYD['Composite_date'][i]
    
    if c != 65535:    #because nd is 65535    
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(True)
    elif (compo[i]==compo[i-1]):
        same_compo.append(False)
    else:
        same_compo.append(True)
            
ref_MYD['Composite_date'] = compo   
ref_MYD = ref_MYD[same_compo]

In [7]:
# Creating MXD

ref_MXD = pd.DataFrame()

# Blend MOD and MYD as function of composite_date for each location
locations = sorted(list(set(ref_MYD.index)))
for l in locations:
    
    MOD = ref_MOD.loc[l]
    MOD = MOD.reset_index()
    MOD = MOD.set_index('Composite_date')
    
    MYD = ref_MYD.loc[l]
    MYD = MYD.reset_index()
    MYD = MYD.set_index('Composite_date')
    
    MXD = pd.concat([MOD, MYD]).sort_index()
    MXD = MXD.reset_index()
    MXD = MXD.set_index('ID')
    
    ref_MXD = pd.concat([ref_MXD, MXD])

# Remove values with same composite_date (keep max value)
same_compo = []
for i in range(len(ref_MXD)):    
    if (i==0):
        same_compo.append(True)
    elif (ref_MXD['Composite_date'][i]==ref_MXD['Composite_date'][i-1]):
        if compute_ndvi_float(ref_MXD['B1'][i-1],ref_MXD['B2'][i-1]) > compute_ndvi_float(ref_MXD['B1'][i],ref_MXD['B2'][i]): 
            same_compo.pop()
            same_compo.append(False)
            same_compo.append(True)
        else:
            same_compo.append(False)
    else:
        same_compo.append(True)
        
ref_MXD = ref_MXD[same_compo]

NameError: name 'compute_ndvi_float' is not defined